In [ ]:
import sys
sys.path.append('../')

In [ ]:
from dvalib import yolo
import random,glob,os
import numpy as np
from keras import backend as K
from keras.models import load_model
from PIL import Image
from dvalib.yolo.keras_yolo import yolo_eval, yolo_head

In [ ]:
path = '/Users/aub3/Dropbox/DeepVideoAnalytics/dvalib/yolo/'
args = {
    'model_path':'{}/model_data/yolo.h5'.format(path),
    'anchors_path': '{}/model_data/yolo_anchors.txt'.format(path),
    'classes_path': '{}/model_data/coco_classes.txt'.format(path),
    'test_path': '{}/images'.format(path),
    'output_path': '{}/images/out'.format(path),
    'score_threshold': 0.3,
    'iou_threshold': 0.5,
}

In [ ]:
model_path = os.path.expanduser(args['model_path'])
anchors_path = os.path.expanduser(args['anchors_path'])
classes_path = os.path.expanduser(args['classes_path'])
test_path = os.path.expanduser(args['test_path'])
output_path = os.path.expanduser(args['output_path'])
sess = K.get_session()
class_names = [c.strip() for c in file(classes_path)]
anchors = np.array( [float(x) for x in file(anchors_path).read().split(',') if x.strip()]).reshape(-1, 2)
yolo_model = load_model(model_path)
num_classes,num_anchors = len(class_names), len(anchors)
model_output_channels = yolo_model.layers[-1].output_shape[-1]
assert model_output_channels == num_anchors * (num_classes + 5)
model_image_size = yolo_model.layers[0].input_shape[1:3]
is_fixed_size = model_image_size != (None, None)
hsv_tuples = [(x / len(class_names), 1., 1.)for x in range(len(class_names))]
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
input_image_shape = K.placeholder(shape=(2, ))
boxes, scores, classes = yolo_eval(yolo_outputs,input_image_shape,score_threshold=args['score_threshold'],iou_threshold=args['iou_threshold'])

In [ ]:
for image_file in glob.glob(test_path+"/*.jpg"):
    image = Image.open(os.path.join(test_path, image_file))
    if is_fixed_size:  # TODO: When resizing we can use minibatch input.
        resized_image = image.resize(
            tuple(reversed(model_image_size)), Image.BICUBIC)
        image_data = np.array(resized_image, dtype='float32')
    else:
        new_image_size = (image.width - (image.width % 32),image.height - (image.height % 32))
        resized_image = image.resize(new_image_size, Image.BICUBIC)
        image_data = np.array(resized_image, dtype='float32')
        print(image_data.shape)
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    out_boxes, out_scores, out_classes = sess.run([boxes, scores, classes],feed_dict={yolo_model.input: image_data,input_image_shape: [image.size[1], image.size[0]],K.learning_phase(): 0})
    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        label = '{} {:.2f}'.format(predicted_class, score)
        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))
sess.close()